In [1]:
!cd .. && make dataset && cd notebooks

>>> Downloading and extracting data files...
Data files already downloaded.
>>> OK.



-   An order might have multiple items.
-   Each item might be fulfilled by a distinct seller.
-   All text identifying stores and partners where replaced by the names of Game of Thrones great houses.

![](https://i.imgur.com/HRhd2Y0.png)


In [2]:
import pandas as pd


In [3]:
customers_df = pd.read_csv(
    "../data/raw/olist_customers_dataset.csv",
    dtype={
        # Nominal qualitative data
        "customer_id": "category",
        "customer_unique_id": "category",
        "customer_city": "category",
        "customer_state": "category",
        "customer_zip_code_prefix": "category",
    },
)
geolocation_df = pd.read_csv(
    "../data/raw/olist_geolocation_dataset.csv",
    dtype={
        # Nominal qualitative data
        "geolocation_zip_code_prefix": "category",
        "geolocation_city": "category",
        "geolocation_state": "category",
        # Continuous quantitative data
        "geolocation_lat": float,
        "geolocation_lng": float,
    },
)
order_items_df = pd.read_csv(
    "../data/raw/olist_order_items_dataset.csv",
    dtype={
        # Nominal qualitative data
        "order_id": "category",
        "order_item_id": "category",
        "product_id": "category",
        "seller_id": "category",
        # Date data
        "shipping_limit_date": str,
        # Continuous quantitative data
        "price": float,
        "freight_value": float,
    },
    parse_dates=["shipping_limit_date"],
)
order_payments_df = pd.read_csv(
    "../data/raw/olist_order_payments_dataset.csv",
    dtype={
        # Nominal qualitative data
        "order_id": "category",
        "payment_type": "category",
        # Discrete quantitative data
        "payment_sequential": int,
        "payment_installments": int,
        # Continuous quantitative data
        "payment_value": float,
    },
)
order_reviews_df = pd.read_csv(
    "../data/raw/olist_order_reviews_dataset.csv",
    dtype={
        # Nominal qualitative data
        "review_id": "category",
        "order_id": "category",
        # Discrete quantitative data
        "review_score": int,
        # Text data
        "review_comment_title": str,
        "review_comment_message": str,
        # Date data
        "review_creation_date": str,
        "review_answer_timestamp": str,
    },
    parse_dates=["review_creation_date", "review_answer_timestamp"],
)
orders_df = pd.read_csv(
    "../data/raw/olist_orders_dataset.csv",
    dtype={
        # Nominal qualitative data
        "order_id": "category",
        "customer_id": "category",
        "order_status": "category",
        # Date data
        "order_purchase_timestamp": str,
        "order_approved_at": str,
        "order_delivered_carrier_date": str,
        "order_delivered_customer_date": str,
        "order_estimated_delivery_date": str,
    },
    parse_dates=[
        "order_purchase_timestamp",
        "order_approved_at",
        "order_delivered_carrier_date",
        "order_delivered_customer_date",
        "order_estimated_delivery_date",
    ],
)
products_df = pd.read_csv(
    "../data/raw/olist_products_dataset.csv",
    dtype={
        # Nominal qualitative data
        "product_id": "category",
        "product_category_name": "category",
        # Discrete quantitative data
        # Nullable : https://pandas.pydata.org/pandas-docs/stable/user_guide/gotchas.html#support-for-integer-na
        "product_name_lenght": pd.Int64Dtype(),
        "product_description_lenght": pd.Int64Dtype(),
        "product_photos_qty": pd.Int64Dtype(),
        # Continuous quantitative data
        "product_weight_g": float,
        "product_length_cm": float,
        "product_height_cm": float,
        "product_width_cm": float,
    },
)
sellers_df = pd.read_csv(
    "../data/raw/olist_sellers_dataset.csv",
    dtype={
        # Nominal qualitative data
        "seller_id": "category",
        "seller_city": "category",
        "seller_state": "category",
        "seller_zip_code_prefix": "category",
    },
)
category_translation_df = pd.read_csv(
    "../data/raw/product_category_name_translation.csv"
)


In [17]:
customers_df.describe(include="all")

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
count,99441,99441,99441,99441,99441
unique,99441,96096,14994,4119,27
top,00012a2ce6f8dcda20d059ce98491703,8d50f5eadf50201ccdcedfb9e2ac8455,22790,sao paulo,SP
freq,1,17,142,15540,41746


In [5]:
geolocation_df.describe(include="all")

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
count,1000163,1.000163e+06,1.000163e+06,1000163,1000163
unique,19015,NaN,NaN,8011,27
top,24220,NaN,NaN,sao paulo,SP
freq,1146,NaN,NaN,135800,404268
mean,NaN,-2.117615e+01,-4.639054e+01,NaN,NaN
std,NaN,5.715866e+00,4.269748e+00,NaN,NaN
min,NaN,-3.660537e+01,-1.014668e+02,NaN,NaN
25%,NaN,-2.360355e+01,-4.857317e+01,NaN,NaN
50%,NaN,-2.291938e+01,-4.663788e+01,NaN,NaN
75%,NaN,-1.997962e+01,-4.376771e+01,NaN,NaN


In [19]:
merged_customers_geolocation_df = (
    customers_df.merge(
        geolocation_df,
        how="left",
        left_on="customer_zip_code_prefix",
        right_on="geolocation_zip_code_prefix",
        validate="m:m",
    )
    .drop(
        columns=[
            "geolocation_zip_code_prefix",
            "geolocation_city",
            "geolocation_state",
        ],
    )
    .rename(
        columns={
            "geolocation_lat": "customer_lat",
            "geolocation_lng": "customer_lng",
        },
    )
    .drop_duplicates()
    .groupby("customer_id")
    .agg(
        {
            "customer_lat": {"describe"},
            "customer_lng": {"describe"},
        }
    )
    .reset_index()
)

merged_customers_geolocation_df.columns = [
    "_".join(a).rstrip("_")
    for a in merged_customers_geolocation_df.columns.to_flat_index()
]

merged_customers_geolocation_df.head()


,customer_id,customer_lat_describe_count,customer_lat_describe_mean,customer_lat_describe_std,customer_lat_describe_min,customer_lat_describe_25%,customer_lat_describe_50%,customer_lat_describe_75%,customer_lat_describe_max,customer_lng_describe_count,customer_lng_describe_mean,customer_lng_describe_std,customer_lng_describe_min,customer_lng_describe_25%,customer_lng_describe_50%,customer_lng_describe_75%,customer_lng_describe_max
0,00012a2ce6f8dcda20d059ce98491703,75.0,-23.499149,0.003785,-23.508027,-23.502090,-23.499387,-23.495892,-23.491469,75.0,-46.767812,0.002348,-46.772523,-46.769529,-46.767180,-46.766522,-46.762668
1,000161a058600d5901f007fab4c27140,87.0,-20.469287,0.019544,-20.477947,-20.473852,-20.472342,-20.470340,-20.341986,87.0,-45.123459,0.009574,-45.156453,-45.126972,-45.123286,-45.117394,-45.107222
2,0001fd6190edaaf884bcaf3d49edf079,170.0,-18.705783,0.022651,-18.823944,-18.712332,-18.707588,-18.700356,-18.494666,170.0,-40.404736,0.040401,-40.756034,-40.402403,-40.398626,-40.395090,-40.385265
3,0002414f95344307404f0ace7a26f1d5,1.0,-21.770599,NaN,-21.770599,-21.770599,-21.770599,-21.770599,-21.770599,1.0,-43.355230,NaN,-43.355230,-43.355230,-43.355230,-43.355230,-43.355230
4,000379cdec625522490c315e70c7a9fb,70.0,-23.738685,0.002170,-23.743357,-23.740622,-23.738425,-23.736918,-23.735298,70.0,-46.689358,0.001307,-46.692461,-46.690310,-46.689274,-46.688569,-46.686224


In [20]:
merged_customers_geolocation_df.describe(include="all")

,customer_id,customer_lat_describe_count,customer_lat_describe_mean,customer_lat_describe_std,customer_lat_describe_min,customer_lat_describe_25%,customer_lat_describe_50%,customer_lat_describe_75%,customer_lat_describe_max,customer_lng_describe_count,customer_lng_describe_mean,customer_lng_describe_std,customer_lng_describe_min,customer_lng_describe_25%,customer_lng_describe_50%,customer_lng_describe_75%,customer_lng_describe_max
count,99441,99441.000000,99163.000000,98890.000000,99163.000000,99163.000000,99163.000000,99163.000000,99163.000000,99441.000000,99163.000000,98890.000000,99163.000000,99163.000000,99163.000000,99163.000000,99163.000000
unique,99441,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,00012a2ce6f8dcda20d059ce98491703,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,101.279794,-21.190859,0.022059,-21.235358,-21.199067,-21.193142,-21.186358,-21.129429,101.279794,-46.175518,0.023687,-46.240568,-46.182739,-46.176115,-46.168918,-46.118771
std,NaN,100.955714,5.609020,0.412477,5.601361,5.601780,5.606745,5.616676,5.714571,100.955714,4.056742,0.362538,4.127783,4.058852,4.057044,4.061781,4.099885
min,NaN,0.000000,-33.689824,0.000024,-36.605374,-33.691625,-33.690729,-33.687725,-33.683447,0.000000,-72.668821,0.000002,-101.466766,-99.229782,-72.670621,-72.665230,-72.456263
25%,NaN,36.000000,-23.589332,0.003080,-23.602547,-23.591624,-23.589494,-23.587122,-23.577656,36.000000,-48.097915,0.003379,-48.157640,-48.106947,-48.097553,-48.094882,-48.048983
50%,NaN,70.000000,-22.924854,0.005427,-22.953606,-22.928771,-22.924820,-22.921836,-22.898404,70.000000,-46.630400,0.005865,-46.643349,-46.632744,-46.630564,-46.629004,-46.612833
75%,NaN,131.000000,-20.139309,0.010263,-20.205982,-20.146553,-20.140688,-20.133579,-20.047075,131.000000,-43.597342,0.011438,-43.696931,-43.606841,-43.601209,-43.593431,-43.473032


In [4]:
sellers_df.describe(include="all")

,seller_id,seller_zip_code_prefix,seller_city,seller_state
count,3095,3095,3095,3095
unique,3095,2246,611,23
top,0015a82c2db000af6aaaf3ae2ecb0532,14940,sao paulo,SP
freq,1,49,694,1849


In [35]:
merged_sellers_geolocation_df = (
    sellers_df.merge(
        geolocation_df,
        how="left",
        left_on="seller_zip_code_prefix",
        right_on="geolocation_zip_code_prefix",
        validate="m:m",
    )
    .drop(
        columns=[
            "geolocation_zip_code_prefix",
            "geolocation_city",
            "geolocation_state",
        ],
    )
    .rename(
        columns={
            "geolocation_lat": "seller_lat",
            "geolocation_lng": "seller_lng",
        },
    )
    .drop_duplicates()
    .groupby("seller_id")
    .agg(
        {
            "seller_lat": {"describe"},
            "seller_lng": {"describe"},
        }
    )
    .reset_index()
)

merged_sellers_geolocation_df.columns = [
    "_".join(a).rstrip("_")
    for a in merged_sellers_geolocation_df.columns.to_flat_index()
]

merged_sellers_geolocation_df.head()


,seller_id,seller_lat_describe_count,seller_lat_describe_mean,seller_lat_describe_std,seller_lat_describe_min,seller_lat_describe_25%,seller_lat_describe_50%,seller_lat_describe_75%,seller_lat_describe_max,seller_lng_describe_count,seller_lng_describe_mean,seller_lng_describe_std,seller_lng_describe_min,seller_lng_describe_25%,seller_lng_describe_50%,seller_lng_describe_75%,seller_lng_describe_max
0,0015a82c2db000af6aaaf3ae2ecb0532,139.0,-23.640101,0.008030,-23.655752,-23.646495,-23.639742,-23.633525,-23.625599,139.0,-46.542293,0.004266,-46.549683,-46.546024,-46.543113,-46.538375,-46.532709
1,001cca7ae9ae17fb1caed9dfb1094831,43.0,-20.279224,0.026011,-20.365605,-20.276193,-20.271105,-20.264805,-20.254806,43.0,-40.411514,0.010701,-40.431168,-40.417310,-40.415232,-40.402785,-40.390723
2,001e6ad469a905060d959994f1b41e4f,39.0,-22.872054,0.002925,-22.879228,-22.874012,-22.872413,-22.870048,-22.865654,39.0,-43.027422,0.005532,-43.036859,-43.033114,-43.027384,-43.021982,-43.019154
3,002100f778ceb8431b7a1020ff7ab48f,239.0,-20.528419,0.010494,-20.550990,-20.535163,-20.529230,-20.522191,-20.503089,239.0,-47.411287,0.005181,-47.429739,-47.414605,-47.410958,-47.407733,-47.400131
4,003554e2dce176b5555353e4f3555ac8,34.0,-16.638074,0.027190,-16.671984,-16.644348,-16.640445,-16.636768,-16.495225,34.0,-49.281838,0.040235,-49.422864,-49.277010,-49.274016,-49.270789,-49.198478


In [21]:
merged_sellers_geolocation_df.describe(include="all")

,seller_id,seller_lat_describe_count,seller_lat_describe_mean,seller_lat_describe_std,seller_lat_describe_min,seller_lat_describe_25%,seller_lat_describe_50%,seller_lat_describe_75%,seller_lat_describe_max,seller_lng_describe_count,seller_lng_describe_mean,seller_lng_describe_std,seller_lng_describe_min,seller_lng_describe_25%,seller_lng_describe_50%,seller_lng_describe_75%,seller_lng_describe_max
count,3095,3095.000000,3088.000000,3078.000000,3088.000000,3088.000000,3088.000000,3088.000000,3088.000000,3095.000000,3088.000000,3078.000000,3088.000000,3088.000000,3088.000000,3088.000000,3088.000000
unique,3095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,0015a82c2db000af6aaaf3ae2ecb0532,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,98.185784,-23.045553,0.015335,-23.087441,-23.050504,-23.046898,-23.043379,-22.991709,98.185784,-47.362669,0.015427,-47.408041,-47.367352,-47.363528,-47.359283,-47.307627
std,NaN,91.327465,3.175562,0.171278,3.193019,3.176253,3.175971,3.176096,3.230338,91.327465,2.805769,0.120017,2.826569,2.806991,2.806803,2.806702,2.829224
min,NaN,0.000000,-32.075303,0.000025,-36.605374,-32.080347,-32.073840,-32.068231,-32.061249,0.000000,-67.813790,0.000158,-67.884719,-67.817821,-67.812015,-67.809441,-67.787028
25%,NaN,35.000000,-23.649123,0.002936,-23.672006,-23.652311,-23.649395,-23.647213,-23.637305,35.000000,-49.071486,0.003086,-49.105390,-49.074348,-49.071596,-49.067753,-49.027836
50%,NaN,71.000000,-23.490016,0.005122,-23.505149,-23.490761,-23.488946,-23.486342,-23.474773,71.000000,-46.792304,0.005380,-46.807235,-46.798238,-46.796434,-46.791904,-46.770130
75%,NaN,130.000000,-22.273658,0.008398,-22.336772,-22.280433,-22.276791,-22.267385,-22.219184,130.000000,-46.525270,0.009009,-46.543165,-46.527908,-46.525632,-46.523028,-46.503967


In [7]:
order_items_df.describe(include="all", datetime_is_numeric=True)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
count,112650,112650,112650,112650,112650,112650.000000,112650.000000
unique,98666,21,32951,3095,NaN,NaN,NaN
top,8272b63d03f5f79c56e9e4120aec44ef,1,aca2eb7d00ea1a7b8ebd4e68314663af,6560211a19b47992c3666cc44a7e94c0,NaN,NaN,NaN
freq,21,98666,527,2033,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,2018-01-07 15:36:52.192685312,120.653739,19.990320
min,NaN,NaN,NaN,NaN,2016-09-19 00:15:34,0.850000,0.000000
25%,NaN,NaN,NaN,NaN,2017-09-20 20:57:27.500000,39.900000,13.080000
50%,NaN,NaN,NaN,NaN,2018-01-26 13:59:35,74.990000,16.260000
75%,NaN,NaN,NaN,NaN,2018-05-10 14:34:00.750000128,134.900000,21.150000
max,NaN,NaN,NaN,NaN,2020-04-09 22:35:08,6735.000000,409.680000


In [8]:
merged_order_items_sellers_df = order_items_df.merge(
    merged_sellers_geolocation_df,
    how="left",
    left_on="seller_id",
    right_on="seller_id",
    validate="m:1",
)

merged_order_items_sellers_df.head()


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,seller_lat_describe_count,seller_lat_describe_mean,seller_lat_describe_std,...,seller_lat_describe_75%,seller_lat_describe_max,seller_lng_describe_count,seller_lng_describe_mean,seller_lng_describe_std,seller_lng_describe_min,seller_lng_describe_25%,seller_lng_describe_50%,seller_lng_describe_75%,seller_lng_describe_max
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,45.0,-22.497188,0.004655,...,-22.496514,-22.485553,45.0,-44.127324,0.005260,-44.144711,-44.130086,-44.125997,-44.124034,-44.117520
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,29.0,-23.565828,0.002767,...,-23.564225,-23.561320,29.0,-46.519217,0.001210,-46.521540,-46.519931,-46.519215,-46.518548,-46.515620
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,61.0,-22.262802,0.029469,...,-22.265723,-22.183679,61.0,-46.170735,0.028730,-46.239752,-46.167895,-46.165743,-46.161760,-46.059739
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,326.0,-20.553651,0.009445,...,-20.545767,-20.532984,326.0,-47.387145,0.011716,-47.409303,-47.396940,-47.387749,-47.377012,-47.360106
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,108.0,-22.929583,0.004802,...,-22.926346,-22.916448,108.0,-53.135750,0.005851,-53.148630,-53.138908,-53.136361,-53.132609,-53.116224


In [9]:
products_df.describe(include="all")

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
count,32951,32341,32341.000000,32341.000000,32341.000000,32949.000000,32949.000000,32949.000000,32949.000000
unique,32951,73,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,00066f42aeeb9f3007548bb9d3f33c38,cama_mesa_banho,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,3029,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,48.476949,771.495285,2.188986,2276.472488,30.815078,16.937661,23.196728
std,NaN,NaN,10.245741,635.115225,1.736766,4282.038731,16.914458,13.637554,12.079047
min,NaN,NaN,5.000000,4.000000,1.000000,0.000000,7.000000,2.000000,6.000000
25%,NaN,NaN,42.000000,339.000000,1.000000,300.000000,18.000000,8.000000,15.000000
50%,NaN,NaN,51.000000,595.000000,1.000000,700.000000,25.000000,13.000000,20.000000
75%,NaN,NaN,57.000000,972.000000,3.000000,1900.000000,38.000000,21.000000,30.000000


In [13]:
merged_order_items_sellers_products_df = merged_order_items_sellers_df.merge(
    products_df,
    how="left",
    left_on="product_id",
    right_on="product_id",
    validate="m:1",
)

merged_order_items_sellers_products_df.head()


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,seller_lat_describe_count,seller_lat_describe_mean,seller_lat_describe_std,...,seller_lng_describe_75%,seller_lng_describe_max,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,45.0,-22.497188,0.004655,...,-44.124034,-44.117520,cool_stuff,58,598,4,650.0,28.0,9.0,14.0
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,29.0,-23.565828,0.002767,...,-46.518548,-46.515620,pet_shop,56,239,2,30000.0,50.0,30.0,40.0
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,61.0,-22.262802,0.029469,...,-46.161760,-46.059739,moveis_decoracao,59,695,2,3050.0,33.0,13.0,33.0
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,326.0,-20.553651,0.009445,...,-47.377012,-47.360106,perfumaria,42,480,1,200.0,16.0,10.0,15.0
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,108.0,-22.929583,0.004802,...,-53.132609,-53.116224,ferramentas_jardim,59,409,1,3750.0,35.0,40.0,30.0


In [15]:
merged_order_items_sellers_products_df.describe(include="all", datetime_is_numeric=True)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,seller_lat_describe_count,seller_lat_describe_mean,seller_lat_describe_std,...,seller_lng_describe_75%,seller_lng_describe_max,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
count,112650,112650,112650,112650,112650,112650.000000,112650.000000,112650.000000,112397.000000,112058.000000,...,112397.000000,112397.000000,111047,111047.000000,111047.000000,111047.000000,112632.000000,112632.000000,112632.000000,112632.000000
unique,98666,21,32951,3095,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,73,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,8272b63d03f5f79c56e9e4120aec44ef,1,aca2eb7d00ea1a7b8ebd4e68314663af,6560211a19b47992c3666cc44a7e94c0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,cama_mesa_banho,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,21,98666,527,2033,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,11115,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,2018-01-07 15:36:52.192685312,120.653739,19.990320,106.343773,-22.797781,0.010802,...,-47.232204,-47.203061,NaN,48.775978,787.867029,2.209713,2093.672047,30.153669,16.593766,22.996546
min,NaN,NaN,NaN,NaN,2016-09-19 00:15:34,0.850000,0.000000,0.000000,-32.075303,0.000025,...,-67.809441,-67.787028,NaN,5.000000,4.000000,1.000000,0.000000,7.000000,2.000000,6.000000
25%,NaN,NaN,NaN,NaN,2017-09-20 20:57:27.500000,39.900000,13.080000,37.000000,-23.608066,0.002903,...,-48.824369,-48.809349,NaN,42.000000,348.000000,1.000000,300.000000,18.000000,8.000000,15.000000
50%,NaN,NaN,NaN,NaN,2018-01-26 13:59:35,74.990000,16.260000,78.000000,-23.424361,0.005129,...,-46.740481,-46.714952,NaN,52.000000,603.000000,1.000000,700.000000,25.000000,13.000000,20.000000
75%,NaN,NaN,NaN,NaN,2018-05-10 14:34:00.750000128,134.900000,21.150000,151.000000,-21.757225,0.007513,...,-46.518548,-46.508612,NaN,57.000000,987.000000,3.000000,1800.000000,38.000000,20.000000,30.000000
max,NaN,NaN,NaN,NaN,2020-04-09 22:35:08,6735.000000,409.680000,746.000000,-2.503367,6.537234,...,-34.852073,-34.847687,NaN,76.000000,3992.000000,20.000000,40425.000000,105.000000,105.000000,118.000000


In [23]:
orders_df.describe(include="all", datetime_is_numeric=True)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
count,99441,99441,99441,99441,99281,97658,96476,99441
unique,99441,99441,8,NaN,NaN,NaN,NaN,NaN
top,000229ec398224ef6ca0657da4fc703e,000161a058600d5901f007fab4c27140,delivered,NaN,NaN,NaN,NaN,NaN
freq,1,1,96478,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,2017-12-31 08:43:12.776581120,2017-12-31 18:35:24.098800128,2018-01-04 21:49:48.138278656,2018-01-14 12:09:19.035542272,2018-01-24 03:08:37.730111232
min,NaN,NaN,NaN,2016-09-04 21:15:19,2016-09-15 12:16:38,2016-10-08 10:34:01,2016-10-11 13:46:32,2016-09-30 00:00:00
25%,NaN,NaN,NaN,2017-09-12 14:46:19,2017-09-12 23:24:16,2017-09-15 22:28:50.249999872,2017-09-25 22:07:22.249999872,2017-10-03 00:00:00
50%,NaN,NaN,NaN,2018-01-18 23:04:36,2018-01-19 11:36:13,2018-01-24 16:10:58,2018-02-02 19:28:10.500000,2018-02-15 00:00:00
75%,NaN,NaN,NaN,2018-05-04 15:42:16,2018-05-04 20:35:10,2018-05-08 13:37:45,2018-05-15 22:48:52.249999872,2018-05-25 00:00:00
max,NaN,NaN,NaN,2018-10-17 17:30:18,2018-09-03 17:40:06,2018-09-11 19:48:28,2018-10-17 13:22:46,2018-11-12 00:00:00


In [27]:
order_payments_df.describe(include="all")

,order_id,payment_sequential,payment_type,payment_installments,payment_value
count,103886,103886.000000,103886,103886.000000,103886.000000
unique,99440,NaN,5,NaN,NaN
top,fa65dad1b0e818e3ccc5cb0e39231352,NaN,credit_card,NaN,NaN
freq,29,NaN,76795,NaN,NaN
mean,NaN,1.092679,NaN,2.853349,154.100380
std,NaN,0.706584,NaN,2.687051,217.494064
min,NaN,1.000000,NaN,0.000000,0.000000
25%,NaN,1.000000,NaN,1.000000,56.790000
50%,NaN,1.000000,NaN,1.000000,100.000000
75%,NaN,1.000000,NaN,4.000000,171.837500


In [32]:
order_payments_df.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45
5,298fcdf1f73eb413e4d26d01b25bc1cd,1,credit_card,2,96.12
6,771ee386b001f06208a7419e4fc1bbd7,1,credit_card,1,81.16
7,3d7239c394a212faae122962df514ac7,1,credit_card,3,51.84
8,1f78449c87a54faf9e96e88ba1491fa9,1,credit_card,6,341.09
9,0573b5e23cbd798006520e1d5b4c6714,1,boleto,1,51.95


In [33]:
merged_orders_order_payments_df = (
    orders_df.merge(
        order_payments_df,
        how="left",
        left_on="order_id",
        right_on="order_id",
        validate="1:m",
    )
    .drop_duplicates()
    .groupby("order_id")
    .agg(
        {
            "payment_sequential": {"describe"},
            "payment_type": {"describe"},
            "payment_installments": {"describe", "sum"},
            "payment_value": {"describe", "sum"},
        }
    )
    .reset_index()
)

merged_orders_order_payments_df.columns = [
    "_".join(a).rstrip("_")
    for a in merged_orders_order_payments_df.columns.to_flat_index()
]


merged_orders_order_payments_df.head()


,order_id,payment_sequential_describe_count,payment_sequential_describe_mean,payment_sequential_describe_std,payment_sequential_describe_min,payment_sequential_describe_25%,payment_sequential_describe_50%,payment_sequential_describe_75%,payment_sequential_describe_max,payment_type_describe_count,...,payment_installments_describe_75%,payment_installments_describe_max,payment_value_describe_count,payment_value_describe_mean,payment_value_describe_std,payment_value_describe_min,payment_value_describe_25%,payment_value_describe_50%,payment_value_describe_75%,payment_value_describe_max
0,00010242fe8c5a6d1ba2dd792cb16214,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1,...,2.0,2.0,1.0,72.19,NaN,72.19,72.19,72.19,72.19,72.19
1,00018f77f2f0320c557190d7a144bdd3,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1,...,3.0,3.0,1.0,259.83,NaN,259.83,259.83,259.83,259.83,259.83
2,000229ec398224ef6ca0657da4fc703e,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1,...,5.0,5.0,1.0,216.87,NaN,216.87,216.87,216.87,216.87,216.87
3,00024acbcdf0a6daa1e931b038114c75,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1,...,2.0,2.0,1.0,25.78,NaN,25.78,25.78,25.78,25.78,25.78
4,00042b26cf59d7ce69dfabb4e55b4fd9,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1,...,3.0,3.0,1.0,218.04,NaN,218.04,218.04,218.04,218.04,218.04


In [34]:
merged_orders_order_payments_df.describe(include="all")

,order_id,payment_sequential_describe_count,payment_sequential_describe_mean,payment_sequential_describe_std,payment_sequential_describe_min,payment_sequential_describe_25%,payment_sequential_describe_50%,payment_sequential_describe_75%,payment_sequential_describe_max,payment_type_describe_count,...,payment_installments_describe_75%,payment_installments_describe_max,payment_value_describe_count,payment_value_describe_mean,payment_value_describe_std,payment_value_describe_min,payment_value_describe_25%,payment_value_describe_50%,payment_value_describe_75%,payment_value_describe_max
count,99441,99441.000000,99440.000000,2961.000000,99440.000000,99440.000000,99440.000000,99440.000000,99440.000000,99441.000000,...,99440.000000,99440.000000,99441.000000,99440.000000,2961.000000,99440.000000,99440.000000,99440.000000,99440.000000,99440.000000
unique,99441,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,00010242fe8c5a6d1ba2dd792cb16214,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,1.044700,1.023160,0.853100,1.000805,1.011982,1.023160,1.034337,1.045515,1.044700,...,2.921996,2.930521,1.044700,158.317915,51.782256,157.204047,157.751011,158.290966,158.867553,159.473791
std,NaN,0.381178,0.192639,0.475957,0.028353,0.099380,0.192639,0.287235,0.382177,0.381178,...,2.705809,2.715685,0.381178,219.194549,110.596564,219.039401,218.995932,219.207349,219.655876,220.369654
min,NaN,0.000000,1.000000,0.707107,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,NaN,1.000000,1.000000,0.707107,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,60.190000,10.705597,59.037500,59.620000,60.100000,60.780000,61.070000
50%,NaN,1.000000,1.000000,0.707107,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,2.000000,2.000000,1.000000,103.330000,25.979103,102.700000,102.960000,103.300000,103.590000,104.120000
75%,NaN,1.000000,1.000000,0.707107,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,4.000000,4.000000,1.000000,174.990000,55.520889,174.360000,174.702500,174.990000,175.320000,175.792500


In [37]:
order_reviews_df.describe(include="all", datetime_is_numeric=True)

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
count,100000,100000,100000.000000,11715,41753,100000,100000
unique,99173,99441,NaN,4600,36921,NaN,NaN
top,dbdf1ea31790c8ecfcc6750525661a9b,c88b1d1b157a9999ce368f218a407141,NaN,Recomendo,Muito bom,NaN,NaN
freq,3,3,NaN,426,230,NaN,NaN
mean,NaN,NaN,4.070890,NaN,NaN,2018-01-12 17:58:10.739999744,2018-01-15 21:30:46.371030016
min,NaN,NaN,1.000000,NaN,NaN,2016-10-02 00:00:00,2016-10-07 18:32:28
25%,NaN,NaN,4.000000,NaN,NaN,2017-09-23 00:00:00,2017-09-27 01:19:37.249999872
50%,NaN,NaN,5.000000,NaN,NaN,2018-02-02 00:00:00,2018-02-04 19:31:06.500000
75%,NaN,NaN,5.000000,NaN,NaN,2018-05-15 00:00:00,2018-05-20 11:00:14.500000
max,NaN,NaN,5.000000,NaN,NaN,2018-08-31 00:00:00,2018-10-29 12:27:35


In [43]:
merged_orders_order_payments_reviews_df = (
    merged_orders_order_payments_df.merge(
        order_reviews_df,
        how="left",
        left_on="order_id",
        right_on="order_id",
        validate="1:m",
    )
    .drop_duplicates()
    .groupby("order_id")
    .agg(
        {
            "review_score": {"describe"},
            "review_comment_title": {"describe"},
            "review_comment_message": {"describe"},
            "review_creation_date": {"describe"},
            "review_answer_timestamp": {"describe"},
        }
    )
    .reset_index()
)

merged_orders_order_payments_reviews_df.columns = [
    "_".join(a).rstrip("_")
    for a in merged_orders_order_payments_reviews_df.columns.to_flat_index()
]


merged_orders_order_payments_reviews_df.head()


/home/clement/Workspace/oc_p5/env/lib/python3.9/site-packages/pandas/core/groupby/generic.py:677: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  result = self.apply(lambda x: x.describe(**kwargs))


,order_id,review_score_describe_count,review_score_describe_mean,review_score_describe_std,review_score_describe_min,review_score_describe_25%,review_score_describe_50%,review_score_describe_75%,review_score_describe_max,review_comment_title_describe_count,...,review_creation_date_describe_top,review_creation_date_describe_freq,review_creation_date_describe_first,review_creation_date_describe_last,review_answer_timestamp_describe_count,review_answer_timestamp_describe_unique,review_answer_timestamp_describe_top,review_answer_timestamp_describe_freq,review_answer_timestamp_describe_first,review_answer_timestamp_describe_last
0,00010242fe8c5a6d1ba2dd792cb16214,1.0,5.0,NaN,5.0,5.0,5.0,5.0,5.0,0,...,2017-09-21,1,2017-09-21,2017-09-21,1,1,2017-09-22 10:57:03,1,2017-09-22 10:57:03,2017-09-22 10:57:03
1,00018f77f2f0320c557190d7a144bdd3,1.0,4.0,NaN,4.0,4.0,4.0,4.0,4.0,0,...,2017-05-13,1,2017-05-13,2017-05-13,1,1,2017-05-15 11:34:13,1,2017-05-15 11:34:13,2017-05-15 11:34:13
2,000229ec398224ef6ca0657da4fc703e,1.0,5.0,NaN,5.0,5.0,5.0,5.0,5.0,0,...,2018-01-23,1,2018-01-23,2018-01-23,1,1,2018-01-23 16:06:31,1,2018-01-23 16:06:31,2018-01-23 16:06:31
3,00024acbcdf0a6daa1e931b038114c75,1.0,4.0,NaN,4.0,4.0,4.0,4.0,4.0,0,...,2018-08-15,1,2018-08-15,2018-08-15,1,1,2018-08-15 16:39:01,1,2018-08-15 16:39:01,2018-08-15 16:39:01
4,00042b26cf59d7ce69dfabb4e55b4fd9,1.0,5.0,NaN,5.0,5.0,5.0,5.0,5.0,0,...,2017-03-02,1,2017-03-02,2017-03-02,1,1,2017-03-03 10:54:59,1,2017-03-03 10:54:59,2017-03-03 10:54:59


In [44]:
merged_orders_order_payments_reviews_df.describe(include="all", datetime_is_numeric=True)

,order_id,review_score_describe_count,review_score_describe_mean,review_score_describe_std,review_score_describe_min,review_score_describe_25%,review_score_describe_50%,review_score_describe_75%,review_score_describe_max,review_comment_title_describe_count,...,review_creation_date_describe_top,review_creation_date_describe_freq,review_creation_date_describe_first,review_creation_date_describe_last,review_answer_timestamp_describe_count,review_answer_timestamp_describe_unique,review_answer_timestamp_describe_top,review_answer_timestamp_describe_freq,review_answer_timestamp_describe_first,review_answer_timestamp_describe_last
count,99441,99441.000000,99441.000000,555.000000,99441.000000,99441.000000,99441.000000,99441.000000,99441.000000,99441.000000,...,99441,99441.000000,99441,99441,99441.000000,99441.000000,99441,99441.0,99441,99441
unique,99441,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,00010242fe8c5a6d1ba2dd792cb16214,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,1.005621,4.071235,0.540777,4.069096,4.070167,4.071238,4.072304,4.073370,0.117809,...,2018-01-13 01:34:54.053760768,1.001589,2018-01-13 01:11:46.489274880,2018-01-13 01:53:43.567743488,1.005621,1.005621,2018-01-16 05:10:49.599672320,1.0,2018-01-16 04:43:05.776370176,2018-01-16 05:29:04.553021696
min,NaN,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,...,2016-10-02 00:00:00,1.000000,2016-10-02 00:00:00,2016-10-02 00:00:00,1.000000,1.000000,2016-10-07 18:32:28,1.0,2016-10-07 18:32:28,2016-10-07 18:32:28
25%,NaN,1.000000,4.000000,0.000000,4.000000,4.000000,4.000000,4.000000,4.000000,0.000000,...,2017-09-23 00:00:00,1.000000,2017-09-23 00:00:00,2017-09-23 00:00:00,1.000000,1.000000,2017-09-27 13:09:54,1.0,2017-09-27 13:09:54,2017-09-27 13:09:54
50%,NaN,1.000000,5.000000,0.000000,5.000000,5.000000,5.000000,5.000000,5.000000,0.000000,...,2018-02-02 00:00:00,1.000000,2018-02-02 00:00:00,2018-02-02 00:00:00,1.000000,1.000000,2018-02-05 02:58:42,1.0,2018-02-05 01:24:03,2018-02-05 05:05:53
75%,NaN,1.000000,5.000000,0.707107,5.000000,5.000000,5.000000,5.000000,5.000000,0.000000,...,2018-05-16 00:00:00,1.000000,2018-05-16 00:00:00,2018-05-16 00:00:00,1.000000,1.000000,2018-05-20 13:14:09,1.0,2018-05-20 13:11:26,2018-05-20 13:14:09
max,NaN,3.000000,5.000000,2.828427,5.000000,5.000000,5.000000,5.000000,5.000000,2.000000,...,2018-08-31 00:00:00,2.000000,2018-08-31 00:00:00,2018-08-31 00:00:00,3.000000,3.000000,2018-10-29 12:27:35,1.0,2018-10-29 12:27:35,2018-10-29 12:27:35
